In [1]:
import time
import xml.etree.ElementTree as ET
import os
import pickle
import gzip
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from IPython.display import HTML, display
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.patches import Patch
from tqdm import tqdm

import os
import pickle
import gzip
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

display(HTML("<style>.container { width:100% !important; }</style>"))

# from dotenv import load_dotenv
import os

# load_dotenv()
# API_KEY = os.getenv("NCBI_KEY")

# Preprocess for figure 2.
- Get the paper list
- eseach on PubMed
- make a table.

In [2]:
def search_pubmed(query, max_results=None):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {"db": "pubmed", "term": query, "retmax": max_results}

    if max_results:
        params["retmax"] = max_results

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        pmids = [id_elem.text for id_elem in root.findall("IdList/Id")]
        return pmids
    elif response.status_code == 429:
        time.sleep(2)
        return search_pubmed(query, 300)
    else:
        print("Error: Unable to fetch data from PubMed")
        return []

In [3]:
def fetch_details(pmids):
    ids = ",".join(pmids)
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {"db": "pubmed", "retmode": "xml", "id": ids}

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print("Error: Unable to fetch details from PubMed")
        return None

    return response.content

In [4]:
def parse_pubmed_efetch(xml_data):
    root = ET.fromstring(xml_data)

    articles = []
    for article in root.findall(".//PubmedArticle"):
        title = article.find(".//ArticleTitle").text

        journal_title = article.find(".//Journal/Title").text

        authors = []
        for author in article.findall(".//Author"):
            lastname = author.find("LastName")
            firstname = author.find("ForeName")
            if lastname is not None and firstname is not None:
                authors.append(f"{firstname.text} {lastname.text}")

        pmcid = None
        for article_id in article.findall(".//ArticleId"):
            if article_id.get("IdType") == "pmc":
                pmcid = article_id.text
                break

        articles.append(
            {
                "title": title,
                "journal": journal_title,
                "authors": authors,
                "pmcid": pmcid,
            }
        )

    return articles

In [6]:
attention = pd.read_csv("ACs.csv.gz", index_col=0)
attention

/tmp/ipykernel_481018/546872600.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  attention = pd.read_csv("ACs.csv.gz", index_col=0)


,740,752,755,757,762,1390,1895,3053,3061,3088,...,ZNF788,ZNF804A,ZNF808,ZNF826P,ZNF83,ZNF845,ZNF888,ZNF91,ZNF969P,ZYG11A
740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002500,0.002666,0.002183,0.003051,0.002470,0.002261,0.002498,0.002277,0.002462,0.002226
752,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001998,0.002298,0.001966,0.002144,0.002220,0.002056,0.001954,0.001874,0.002194,0.002454
755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001891,0.002090,0.001900,0.002017,0.002072,0.001958,0.001986,0.001861,0.002080,0.002115
757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002483,0.002611,0.002215,0.003082,0.002460,0.002284,0.002560,0.002326,0.002449,0.002174
762,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002122,0.002432,0.002031,0.002313,0.002295,0.002119,0.002010,0.001945,0.002261,0.002532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF845,0.001425,0.001222,0.000878,0.001430,0.001094,0.001242,0.001341,0.000982,0.001028,0.001151,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZNF888,0.000906,0.001191,0.000705,0.000796,0.001130,0.000958,0.000676,0.000691,0.000706,0.001196,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZNF91,0.001107,0.001330,0.000995,0.001132,0.001315,0.001244,0.001138,0.000951,0.001046,0.001479,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZNF969P,0.001743,0.001298,0.000904,0.001548,0.001190,0.001266,0.001257,0.000976,0.000994,0.001327,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
n_drugs = 976
n_cells = 59
att_dg =  attention.iloc[:n_drugs, n_drugs + n_cells:]
conv = dict(pd.read_csv('nsc_cid_smiles_class_name.csv', usecols=['NSC', 'NAME']).values)

In [8]:
att_dg = att_dg[np.sign(att_dg).sum(axis=1) != 0]
att_dg

,A2M,AASS,ABCA1,ABCB1,ABCC1,ABCC2,ABCC3,ABCC5,ABCG2,ABI3BP,...,ZNF788,ZNF804A,ZNF808,ZNF826P,ZNF83,ZNF845,ZNF888,ZNF91,ZNF969P,ZYG11A
740,0.002725,0.001979,0.002368,0.001838,0.002547,0.002280,0.001863,0.002173,0.002347,0.003436,...,0.002500,0.002666,0.002183,0.003051,0.002470,0.002261,0.002498,0.002277,0.002462,0.002226
752,0.001655,0.002335,0.002387,0.002293,0.002014,0.001788,0.001681,0.001785,0.001985,0.002365,...,0.001998,0.002298,0.001966,0.002144,0.002220,0.002056,0.001954,0.001874,0.002194,0.002454
755,0.001754,0.002084,0.002214,0.001920,0.002042,0.001964,0.001735,0.001855,0.002014,0.002166,...,0.001891,0.002090,0.001900,0.002017,0.002072,0.001958,0.001986,0.001861,0.002080,0.002115
757,0.002839,0.001932,0.002332,0.001793,0.002561,0.002318,0.001932,0.002197,0.002351,0.003501,...,0.002483,0.002611,0.002215,0.003082,0.002460,0.002284,0.002560,0.002326,0.002449,0.002174
762,0.001832,0.002377,0.002434,0.002331,0.002080,0.001818,0.001701,0.001865,0.002030,0.002518,...,0.002122,0.002432,0.002031,0.002313,0.002295,0.002119,0.002010,0.001945,0.002261,0.002532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811429,0.002525,0.002350,0.002435,0.002393,0.002283,0.002301,0.002034,0.002278,0.002283,0.002516,...,0.002198,0.002391,0.002280,0.002472,0.002327,0.002301,0.002355,0.002282,0.002314,0.002434
812926,0.002242,0.002380,0.002411,0.002884,0.002273,0.002172,0.001901,0.002281,0.002318,0.002293,...,0.002269,0.002562,0.002220,0.002381,0.002321,0.002264,0.002251,0.002198,0.002321,0.002560
812927,0.002666,0.001902,0.002298,0.001751,0.002529,0.002264,0.001824,0.002161,0.002335,0.003365,...,0.002521,0.002625,0.002169,0.003005,0.002449,0.002244,0.002479,0.002267,0.002444,0.002165
813488,0.002928,0.002174,0.002445,0.002063,0.002503,0.002478,0.002192,0.002329,0.002391,0.003092,...,0.002305,0.002458,0.002377,0.002830,0.002452,0.002401,0.002648,0.002461,0.002431,0.002331


In [9]:
def load_pickle_auto(file_path):
    if not os.path.exists(file_path):
        return {}
    with open(file_path, "rb") as f:
        first_byte = f.read(2)
        f.seek(0)  # 戻す
        # gzipのシグネチャチェック
        if first_byte == b'\x1f\x8b':
            with gzip.open(f, "rb") as gz:
                return pickle.load(gz)
        else:
            return pickle.load(f)

# キャッシュ読み込み（gzip対応）
search_cache_file = "search_cache.pkl"
details_cache_file = "details_cache.pkl"

search_cache = load_pickle_auto(search_cache_file)
details_cache = load_pickle_auto(details_cache_file)

results = []
pmid_list = []
task_list = []

# 処理すべきすべてのクエリを先に構築
for drug in att_dg.index:
    top_genes = att_dg.loc[drug].sort_values(ascending=False).head(5)
    for rank, gene in enumerate(top_genes.index, start=1):
        d = conv[int(drug)]
        if d == "benzaldehyde (BEN)":
            d = "benzaldehyde"
        elif "Derivative" in d:
            d = d[:-11]
        elif "mesylate" in d:
            d = d[:-9]
        if gene == "ADA":
            gene = "adenosine deaminase"
        query_key = f"{d} AND {gene}"
        task_list.append((drug, d, gene, query_key, rank))

# 並列で処理する関数
def process_query(task):
    drug, drug_name, gene, query_key, rank = task

    # PubMed検索（キャッシュ付き + 空チェック）
    if query_key in search_cache:
        pmids = search_cache[query_key]
    else:
        pmids = search_pubmed(query_key, max_results=300)
        if pmids:  # 空でなければキャッシュ
            search_cache[query_key] = pmids
        else:
            return [drug, drug_name, gene, 0, 0, rank]

    pmid_list.append([drug, gene, pmids, rank])

    # 詳細データ（キャッシュ付き + 空チェック）
    if query_key in details_cache:
        parsed = details_cache[query_key]
    else:
        details = fetch_details(pmids)
        if not details:
            return [drug, drug_name, gene, len(pmids), 0, rank]
        parsed = parse_pubmed_efetch(details.decode("utf-8"))
        if parsed:  # 空でなければキャッシュ
            details_cache[query_key] = parsed
        else:
            return [drug, drug_name, gene, len(pmids), 0, rank]

    found = 1 if parsed else 0
    return [drug, drug_name, gene, len(pmids), found, rank]

# 並列実行
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_query, task) for task in task_list]
    for future in tqdm(as_completed(futures), total=len(futures)):
        results.append(future.result())

# キャッシュ保存（圧縮 pickle 使用）
with open(search_cache_file, "wb") as f:
    pickle.dump(search_cache, f, protocol=pickle.HIGHEST_PROTOCOL)
with gzip.open(details_cache_file, "wb") as f:
    pickle.dump(details_cache, f, protocol=pickle.HIGHEST_PROTOCOL)


  0%|                                                                          | 0/4880 [00:00<?, ?it/s]

Error: Unable to fetch details from PubMed


  1%|▎                                                                | 28/4880 [00:12<37:57,  2.13it/s]

Error: Unable to fetch details from PubMed


  1%|▉                                                                | 69/4880 [00:39<53:05,  1.51it/s]

...

  5%|██▉                                                             | 221/4880 [02:01<38:09,  2.04it/s]

Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed
Error: Unable to fetch details from PubMed


  5%|███▎                                                            | 257/4880 [02:21<37:16,  2.07it/s]

Error: Unable to fetch details from PubMed


  6%|███▋                                                            | 277/4880 [02:32<42:16,  1.81it/s]

KeyboardInterrupt



In [ ]:
results = pd.DataFrame(results)[[0, 1, 2, 3, 5]]
results.columns = ["NSC", "drug", "gene", "num", "rank"]

In [ ]:
results

In [ ]:
np.sign(results['num']).sum()

In [ ]:
results.to_csv("val_PM.csv", index=False)